In [1]:
import time
import pandas as pd
import psycopg2
import datetime
from tqdm import trange
import matplotlib.pyplot as plt 
import matplotlib as mpl 
import seaborn as sns
import math
# %matplotlib inline 
import platform
import numpy as np
from matplotlib import font_manager, rc # plt.rcParams['axes.unicode_minus'] = False 
if platform.system() == 'Darwin': rc('font', family='AppleGothic') 
elif platform.system() == 'Windows': 
    path = "c:/Windows/Fonts/malgun.ttf" 
    font_name = font_manager.FontProperties(fname=path).get_name() 
    rc('font', family=font_name) 
else: 
    print('Unknown system... sorry~~~~')
plt.rcParams["figure.figsize"] = (14,4)
plt.rcParams['lines.linewidth'] = 2
from tqdm import tqdm
from tqdm import trange
from datetime import datetime, timedelta


In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))


In [3]:
# 시계열
from datetime import datetime as dt
from statsmodels.tsa.arima_model import ARIMA
from dateutil.relativedelta import relativedelta
from statsmodels.tsa.api import SimpleExpSmoothing, Holt, ExponentialSmoothing

In [4]:
    # row 생략 없이 출력
pd.set_option('display.max_rows', 1000)
# col 생략 없이 출력
pd.set_option('display.max_columns', 1000)
plt.rcParams["figure.figsize"] = (14,4)
plt.rcParams['lines.linewidth'] = 2
pd.options.display.float_format = '{:.5f}'.format

In [5]:
# 전체 매장 입출판재
total_shop_stock = pd.read_csv("total_shop_stock.csv")
total_shop_stock = total_shop_stock.iloc[ :,1:]

# 매장정보
shop_info_pd = pd.read_csv("shop_info_pd.csv")
shop_info_pd = shop_info_pd.iloc[ :,1:]

# bucket_dwalker 관련 매장입출판재(2019)
bucket_dwalker = pd.read_csv('bucket_dwalker.csv')
bucket_dwalker = bucket_dwalker.iloc[ :,1:]

# bucket_dwalker_v2 관련 매장입출판재
bucket_dwalker_v2 = pd.read_csv('bucket_dwalker_v2.csv')
bucket_dwalker_v2 = bucket_dwalker_v2.iloc[ :,1:]

# 아울렛 상설을 제외한 매장키 
shop_list_pd = pd.read_csv('shop_list.csv')
shop_list_pd = shop_list_pd.iloc[ :,1:]

# 전체 매장 비어있는 주차 채운 데이터 (버킷 디워커 기준)
shop_weeks_buket_df = pd.read_csv('shop_weeks_buket.csv')
shop_weeks_buket_df = shop_weeks_buket_df.iloc[ :,1:]

# 전체 매장 비어있는 주차 채운 데이터 (버킷 디워커 v2 기준)
shop_weeks_buket_v2_df = pd.read_csv('shop_weeks_buket_v2.csv')
shop_weeks_buket_v2_df = shop_weeks_buket_v2_df.iloc[ :,1:]

# 전체 매장 비어있는것 채워넣은 입출판재 합쳐 놓은 것
shop_total_sales = pd.read_csv("shop_total_sales.csv")
shop_total_sales = shop_total_sales.iloc[ :,1:]

In [6]:
# DB 연결 부분
uswer = 'dt.intern'
host = '10.80.0.234'
port = '5439'
dbname = 'fnf'
con = psycopg2.connect(dbname='fnf',host='10.80.0.234', port='5439', user = 'dt.intern', password='DTintn0926')
cur = con.cursor()

In [83]:
# 2019년 버킷디워커 물류 수량
start = time.time()

bk_stock_scsweek = '''

SELECT weekkey ,scskey, week_accept_qty ,week_wsl_takeout_qty, wms_stock_qty ,stock_qty,domestic_sale_qty,csm_takeout_qty,csm_sale_qty
FROM ax.vw_f_scsweek vfs 
WHERE scskey  IN ('X19SDXSH09911BE230','X19SDXSH09911BE235','X19SDXSH09911BE240','X19SDXSH09911BE260','X19SDXSH09911BE265','X19SDXSH09911BE270') AND weekkey BETWEEN To_date('2018-12-31', 'YYYY-MM-DD') AND To_date('2019-12-31', 'YYYY-MM-DD')


'''


cur.execute(bk_stock_scsweek)
bk_stock_scsweek = cur.fetchall()

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

bk_stock_scsweek_pd = pd.DataFrame(bk_stock_scsweek)
bk_stock_scsweek_pd.columns = ['weekkey' ,'scskey', 'week_accept_qty' ,'week_wsl_takeout_qty', 'wms_stock_qty' ,'stock_qty','domestic_sale_qty','csm_takeout_qty','csm_sale_qty']

time : 0.22255897521972656


In [91]:
bk_stock_scsweek_pd['csm_qty'] = (bk_stock_scsweek_pd.csm_takeout_qty - bk_stock_scsweek_pd.csm_sale_qty)

In [99]:
bk_stock_scsweek_pd.to_csv('bk_stock_scsweek_pd.csv')

In [98]:
bk_stock_scsweek_pd

,weekkey,scskey,week_accept_qty,week_wsl_takeout_qty,wms_stock_qty,stock_qty,domestic_sale_qty,csm_takeout_qty,csm_sale_qty,csm_qty
0,2018-12-31,X19SDXSH09911BE230,80,0,0,74,6,80,6,74
1,2019-02-25,X19SDXSH09911BE230,640,3,509,675,1214,1380,1214,166
2,2019-03-25,X19SDXSH09911BE230,1238,-9,1146,2328,2359,3541,2359,1182
3,2019-05-20,X19SDXSH09911BE230,0,1,1117,2639,3798,5320,3798,1522
4,2019-06-03,X19SDXSH09911BE230,0,1,996,2501,3935,5440,3935,1505
5,2019-06-17,X19SDXSH09911BE230,0,0,874,2398,4038,5562,4038,1524
6,2019-06-24,X19SDXSH09911BE230,0,0,893,2366,4070,5543,4070,1473
7,2019-07-01,X19SDXSH09911BE230,0,0,789,2313,4123,5647,4123,1524
8,2019-07-29,X19SDXSH09911BE230,0,0,714,2147,4279,5712,4279,1433
9,2019-08-26,X19SDXSH09911BE230,0,0,572,1922,4509,5859,4509,1350


In [87]:
bk_stock_scsweek_pd.groupby('weekkey').sum()[['wms_stock_qty','stock_qty']].reset_index().to_csv('bk_stock_scsweek_pd.csv')

In [102]:
bk_stock_scsweek_pd.groupby('weekkey').sum()[['wms_stock_qty','stock_qty','csm_takeout_qty','csm_sale_qty']].reset_index()

,weekkey,wms_stock_qty,stock_qty,csm_takeout_qty,csm_sale_qty
0,2018-12-31,4,935,956,25
1,2019-01-07,4,869,956,91
2,2019-01-14,849,1537,1501,813
3,2019-01-21,324,807,2095,1612
4,2019-01-28,84,484,2726,2326
5,2019-02-04,199,-97,2880,3176
6,2019-02-11,266,-241,3820,4327
7,2019-02-18,599,1212,6169,5556
8,2019-02-25,2913,3461,7442,6894
9,2019-03-04,663,3042,10923,8544


In [79]:
# 2019년 버킷디워커 물류 수량
start = time.time()

v2_stock_scsweek = '''

SELECT weekkey ,scskey, week_accept_qty ,week_wsl_takeout_qty, wms_stock_qty ,stock_qty,domestic_sale_qty,csm_takeout_qty,csm_sale_qty
FROM ax.vw_f_scsweek vfs 
WHERE scskey  IN ('X20SDXSHA5011OW230','X20SDXSHA5011OW235','X20SDXSHA5011OW240','X20SDXSHA5011OW260','X20SDXSHA5011OW265','X20SDXSHA5011OW270') AND weekkey BETWEEN To_date('2019-11-18', 'YYYY-MM-DD') AND To_date('2020-12-31', 'YYYY-MM-DD')


'''


cur.execute(v2_stock_scsweek)
v2_stock_scsweek = cur.fetchall()

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

v2_stock_scsweek_pd = pd.DataFrame(v2_stock_scsweek)
v2_stock_scsweek_pd.columns = ['weekkey' ,'scskey', 'week_accept_qty' ,'week_wsl_takeout_qty', 'wms_stock_qty' ,'stock_qty','domestic_sale_qty','csm_takeout_qty','csm_sale_qty']

time : 7.116010427474976


In [82]:
(v2_stock_scsweek_pd.csm_takeout_qty - v2_stock_scsweek_pd.csm_sale_qty ).mean()

895.6553672316384

In [93]:
v2_stock_scsweek_pd['csm_qty'] = (v2_stock_scsweek_pd.csm_takeout_qty - v2_stock_scsweek_pd.csm_sale_qty )

In [96]:
v2_stock_scsweek_pd['csm_qty']

0       343
1       358
2       402
3       414
4       635
5       750
6       626
7       905
8       901
9       929
10      945
11      970
12      994
13      895
14      887
15      875
16      363
17      399
18      520
19      509
20      708
21     1393
22     1344
23     1285
24     1217
25     1264
26     1245
27     1241
28     1239
29     1118
30     1097
31     1104
32      326
33      373
34      422
35      414
36      669
37      903
38      859
39     1050
40     1039
41      986
42     1009
43     1007
44     1030
45      985
46      857
47      837
48      346
49      381
50      460
51      440
52      664
53     1054
54      986
55     1152
56     1097
57     1059
58     1051
59     1031
60     1072
61      977
62      925
63      922
64      348
65      365
66      401
67      382
68      595
69      842
70      804
71     1043
72     1032
73     1033
74     1100
75     1099
76     1093
77     1026
78     1019
79     1020
80      360
81      362
82      411
83  

In [97]:
v2_stock_scsweek_pd.to_csv('v2_stock_scsweek_pd.csv')

In [11]:
v2_stock_scsweek_pd.groupby('weekkey').sum()[['wms_stock_qty','stock_qty']].reset_index().to_csv('v2_stock_scsweek_pd.csv')

In [78]:
v2_stock_scsweek_pd.groupby('weekkey').sum()[['wms_stock_qty','stock_qty']].reset_index()

,weekkey,wms_stock_qty,stock_qty
0,2019-11-18,3224,5199
1,2019-11-25,3350,5436
2,2019-12-02,2611,4849
3,2019-12-09,1941,4277
4,2019-12-16,1087,3516
5,2019-12-23,191,2669
6,2019-12-30,1548,4164
7,2020-01-06,1476,4016
8,2020-01-13,5949,8989
9,2020-01-20,5200,9081


In [15]:
def mae(prediction, correct):
    prediction = np.array(prediction)
    correct = np.array(correct)
    
    difference = correct - prediction
    abs_val = abs(difference)
    
    score = abs_val.mean()
    
    return score

# mae_scorer = make_scorer(mae)
# mae_scorer

In [16]:
def make_ewm_arr(data_frame, span_num):
    arr_ewm = np.array([])
#     for i in data_frame.store_id.unique():
    df_set = data_frame
        # 여기에서 지정하는 span값은 위 수식에서 N에 해당한다.
    arr_ewm = np.concatenate((arr_ewm, df_set.week_sale_qty.ewm(span=span_num).mean().values) )
    
    return arr_ewm

# 재고가 0 인 지점

## 2020년 버킷디워커 v2

In [17]:
shop_weeks_buket_v2_df_dep = shop_weeks_buket_v2_df[shop_weeks_buket_v2_df['type_zone_nm']=='백화점']

In [18]:
shop_weeks_buket_v2_df_dep = shop_weeks_buket_v2_df_dep[shop_weeks_buket_v2_df_dep['shopkey'] != 'X10047']

In [19]:
v2_shopkey_unique =  shop_weeks_buket_v2_df_dep.shopkey.unique()
len(v2_shopkey_unique)

66

In [20]:
v2_scskey_unique =  shop_weeks_buket_v2_df_dep.scskey.unique()
len(v2_scskey_unique)

6

In [21]:
v2_shop_qty = pd.DataFrame()

for i in v2_shopkey_unique:
    uniqe_shop = shop_weeks_buket_v2_df_dep[shop_weeks_buket_v2_df_dep['shopkey']==i]
    for j in v2_scskey_unique:
        uniqe_shop_scskey = uniqe_shop[uniqe_shop['scskey']==j]
        ewm_value = make_ewm_arr(uniqe_shop_scskey, 2)
        uniqe_shop_scskey.insert(11, 'amount_2ewm', ewm_value)
        ewm_value = np.append([0],ewm_value,axis=0)
        uniqe_shop_scskey['amount_2ewm_week'] = ewm_value[:-1]
        for i in range(0,len(ewm_value)):
#             ewm_value[i] = round(ewm_value[i] , 0)+1
            ewm_value[i] = math.ceil(ewm_value[i])+1

        uniqe_shop_scskey['qty'] = ewm_value[:-1]
        v2_shop_qty = pd.concat([v2_shop_qty, uniqe_shop_scskey], axis=0)

<ipython-input-21-7d2f0dd62d4a>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uniqe_shop_scskey['amount_2ewm_week'] = ewm_value[:-1]
<ipython-input-21-7d2f0dd62d4a>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uniqe_shop_scskey['qty'] = ewm_value[:-1]


In [104]:
v2_shop_qty.groupby('weekkey').sum()

,week_takeout_qty,takeout_qty,week_sale_qty,sale_qty,stock_qty,ratio,amount_2ewm,amount_2ewm_week,qty
weekkey,,,,,,,,,
2019-11-18,732,732,98,98,634,2.62746,98.00000,0.00000,396.00000
2019-11-25,249,981,191,289,692,8.12039,167.75000,98.00000,494.00000
2019-12-02,345,1326,257,546,780,15.77649,229.53846,167.75000,634.00000
2019-12-09,298,1624,260,806,818,22.43627,250.10000,229.53846,753.00000
2019-12-16,454,2078,379,1185,893,34.00044,336.38843,250.10000,813.00000
2019-12-23,329,2407,363,1548,859,44.05726,354.15385,336.38843,919.00000
2019-12-30,377,2784,290,1838,946,52.27536,311.36505,354.15385,950.00000
2020-01-06,320,3104,329,2167,937,61.76619,323.12348,311.36505,917.00000
2020-01-13,494,3598,285,2452,1146,69.38468,297.70653,323.12348,930.00000


In [22]:
v2_shop_qty

,weekkey,week_takeout_qty,takeout_qty,week_sale_qty,sale_qty,stock_qty,shopkey,ratio,scskey,shop_nm,shop_nm_sfdc,amount_2ewm,repr_shopcode,type_shop_nm,type_contract_nm,type_zone_nm,region_nm,amount_2ewm_week,qty
0,2019-11-18,1,1,0,0,1,X10056,0.00000,X20SDXSHA5011OW230,갤러리아진주,디스커버리 갤러리아 진주점,0.00000,XH0247,일반매장,백화점특정,백화점,경남,0.00000,1.00000
1,2019-11-25,0,1,0,0,1,X10056,0.00000,X20SDXSHA5011OW230,갤러리아진주,디스커버리 갤러리아 진주점,0.00000,XH0247,일반매장,백화점특정,백화점,경남,0.00000,1.00000
2,2019-12-02,2,3,1,1,2,X10056,0.11111,X20SDXSHA5011OW230,갤러리아진주,디스커버리 갤러리아 진주점,0.69231,XH0247,일반매장,백화점특정,백화점,경남,0.00000,1.00000
3,2019-12-09,0,3,0,1,2,X10056,0.11111,X20SDXSHA5011OW230,갤러리아진주,디스커버리 갤러리아 진주점,0.22500,XH0247,일반매장,백화점특정,백화점,경남,0.69231,2.00000
4,2019-12-16,0,3,0,1,2,X10056,0.11111,X20SDXSHA5011OW230,갤러리아진주,디스커버리 갤러리아 진주점,0.07438,XH0247,일반매장,백화점특정,백화점,경남,0.22500,2.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50499,2020-11-30,0,52,0,44,8,X10038,1.00000,X20SDXSHA5011OW270,롯데창원,디스커버리 롯데 창원점,0.11709,XH0237,일반매장,백화점특정,백화점,경남,0.35127,2.00000
50500,2020-12-07,0,52,0,44,8,X10038,1.00000,X20SDXSHA5011OW270,롯데창원,디스커버리 롯데 창원점,0.03903,XH0237,일반매장,백화점특정,백화점,경남,0.11709,2.00000
50501,2020-12-14,0,52,0,44,8,X10038,1.00000,X20SDXSHA5011OW270,롯데창원,디스커버리 롯데 창원점,0.01301,XH0237,일반매장,백화점특정,백화점,경남,0.03903,2.00000
50502,2020-12-21,0,52,0,44,8,X10038,1.00000,X20SDXSHA5011OW270,롯데창원,디스커버리 롯데 창원점,0.00434,XH0237,일반매장,백화점특정,백화점,경남,0.01301,2.00000


In [23]:
v2_zero = v2_shop_qty[(v2_shop_qty['stock_qty']==0) & (v2_shop_qty['takeout_qty']>0)]

In [24]:
len(v2_zero)

240

In [25]:
len(v2_zero[v2_zero['weekkey'] < '2020-11-01'])

205

In [26]:
# 재고에서 0 이 아닌 부분
len(v2_zero[v2_zero.qty - v2_zero.week_sale_qty>0])

78

In [27]:
# 재고에서 0 인 부분 
len(v2_zero[v2_zero.qty - v2_zero.week_sale_qty<=0])

162

In [28]:
shop_weeks_buket_v2_df_dep

,weekkey,week_takeout_qty,takeout_qty,week_sale_qty,sale_qty,stock_qty,shopkey,ratio,scskey,shop_nm,shop_nm_sfdc,repr_shopcode,type_shop_nm,type_contract_nm,type_zone_nm,region_nm
0,2019-11-18,1,1,0,0,1,X10056,0.00000,X20SDXSHA5011OW230,갤러리아진주,디스커버리 갤러리아 진주점,XH0247,일반매장,백화점특정,백화점,경남
1,2019-11-25,0,1,0,0,1,X10056,0.00000,X20SDXSHA5011OW230,갤러리아진주,디스커버리 갤러리아 진주점,XH0247,일반매장,백화점특정,백화점,경남
2,2019-12-02,2,3,1,1,2,X10056,0.11111,X20SDXSHA5011OW230,갤러리아진주,디스커버리 갤러리아 진주점,XH0247,일반매장,백화점특정,백화점,경남
3,2019-12-09,0,3,0,1,2,X10056,0.11111,X20SDXSHA5011OW230,갤러리아진주,디스커버리 갤러리아 진주점,XH0247,일반매장,백화점특정,백화점,경남
4,2019-12-16,0,3,0,1,2,X10056,0.11111,X20SDXSHA5011OW230,갤러리아진주,디스커버리 갤러리아 진주점,XH0247,일반매장,백화점특정,백화점,경남
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50499,2020-11-30,0,52,0,44,8,X10038,1.00000,X20SDXSHA5011OW270,롯데창원,디스커버리 롯데 창원점,XH0237,일반매장,백화점특정,백화점,경남
50500,2020-12-07,0,52,0,44,8,X10038,1.00000,X20SDXSHA5011OW270,롯데창원,디스커버리 롯데 창원점,XH0237,일반매장,백화점특정,백화점,경남
50501,2020-12-14,0,52,0,44,8,X10038,1.00000,X20SDXSHA5011OW270,롯데창원,디스커버리 롯데 창원점,XH0237,일반매장,백화점특정,백화점,경남
50502,2020-12-21,0,52,0,44,8,X10038,1.00000,X20SDXSHA5011OW270,롯데창원,디스커버리 롯데 창원점,XH0237,일반매장,백화점특정,백화점,경남


In [29]:
frame = shop_weeks_buket_v2_df_dep[:59][['weekkey','stock_qty']]

In [33]:
v2_zero_modi = v2_zero[v2_zero.qty - v2_zero.week_sale_qty<=0].groupby(['weekkey']).count().qty.reset_index()

In [34]:
v2_zero[v2_zero.qty - v2_zero.week_sale_qty<=0].groupby(['weekkey']).count().qty.reset_index().to_csv('v2_zero_modi.csv')

In [35]:
v2_zero_modi_frame = shop_weeks_buket_v2_df_dep[:59][['weekkey','stock_qty']]
v2_zero_modi_frame.stock_qty = 0
for i in range(0,len(v2_zero_modi_frame)):
    for j in range(0,len(v2_zero_modi)):
        if v2_zero_modi_frame.iloc[i].weekkey == v2_zero_modi.iloc[j].weekkey:
            v2_zero_modi_frame.iloc[i] = [v2_zero_modi.iloc[j].weekkey , v2_zero_modi.iloc[j].qty]

In [36]:
v2_zero_modi_frame.to_csv('v2_zero_modi_frame.csv')

In [38]:
v2_zero_ori = v2_zero.groupby(['weekkey']).count().qty.reset_index()

In [39]:
v2_zero_ori_frame = shop_weeks_buket_v2_df_dep[:59][['weekkey','stock_qty']]
v2_zero_ori_frame.stock_qty = 0

for i in range(0,len(v2_zero_ori_frame)):
    for j in range(0,len(v2_zero_ori)):
        if v2_zero_ori_frame.iloc[i].weekkey == v2_zero_ori.iloc[j].weekkey:
            v2_zero_ori_frame.iloc[i] = [v2_zero_ori.iloc[j].weekkey , v2_zero_ori.iloc[j].qty]

In [40]:
v2_zero_ori_frame.to_csv('v2_zero_ori_frame.csv')

In [41]:
v2_merge_outer = pd.merge(v2_zero_ori_frame,v2_zero_modi_frame, how='outer',on='weekkey')
v2_merge_outer.to_csv('v2_merge_outer.csv')

In [42]:
v2_merge_outer.to_csv('v2_merge_outer.csv')

In [43]:
v2_zero[v2_zero.qty - v2_zero.week_sale_qty<=0].groupby(['weekkey','scskey']).count().qty.reset_index()

,weekkey,scskey,qty
0,2019-11-18,X20SDXSHA5011OW230,3
1,2019-11-18,X20SDXSHA5011OW235,5
2,2019-11-18,X20SDXSHA5011OW240,2
3,2019-11-18,X20SDXSHA5011OW260,4
4,2019-11-18,X20SDXSHA5011OW265,4
5,2019-11-18,X20SDXSHA5011OW270,1
6,2019-11-25,X20SDXSHA5011OW230,1
7,2019-11-25,X20SDXSHA5011OW235,7
8,2019-11-25,X20SDXSHA5011OW240,3
9,2019-11-25,X20SDXSHA5011OW260,4


In [44]:
v2_zero[v2_zero.qty - v2_zero.week_sale_qty<=0].groupby(['weekkey']).count().qty.reset_index()

,weekkey,qty
0,2019-11-18,19
1,2019-11-25,22
2,2019-12-02,18
3,2019-12-09,14
4,2019-12-16,12
5,2019-12-23,10
6,2019-12-30,5
7,2020-01-06,9
8,2020-01-13,4
9,2020-01-20,2


In [45]:
v2_zero.groupby(['weekkey']).count().qty.reset_index()

,weekkey,qty
0,2019-11-18,19
1,2019-11-25,23
2,2019-12-02,19
3,2019-12-09,19
4,2019-12-16,13
5,2019-12-23,15
6,2019-12-30,11
7,2020-01-06,11
8,2020-01-13,6
9,2020-01-20,3


In [46]:
v2_zero.groupby(['weekkey']).count().qty.reset_index().to_csv('v2_zero_ori.csv')

In [126]:
shop_weeks_buket_v2_df_dep.groupby(['weekkey']).sum().stock_qty.to_csv('v2_shop_qty.csv')

In [125]:
shop_weeks_buket_v2_df_dep.groupby(['weekkey']).sum().stock_qty.mean()

1750.457627118644

In [122]:
v2_stock_scsweek_pd.groupby(['weekkey']).sum().wms_stock_qty

,weekkey,scskey,week_accept_qty,week_wsl_takeout_qty,wms_stock_qty,stock_qty,domestic_sale_qty,csm_takeout_qty,csm_sale_qty,csm_qty
0,2019-11-25,X20SDXSHA5011OW270,86,0,526,869,96,439,96,343
1,2019-12-02,X20SDXSHA5011OW270,1,0,413,771,195,553,195,358
2,2019-12-30,X20SDXSHA5011OW270,370,0,218,620,716,1118,716,402
3,2020-01-06,X20SDXSHA5011OW270,50,-1,144,558,829,1243,829,414
4,2020-01-20,X20SDXSHA5011OW270,0,0,642,1277,1069,1704,1069,635
5,2020-02-24,X20SDXSHA5011OW270,150,3,70,820,1990,2740,1990,750
6,2020-03-02,X20SDXSHA5011OW270,170,50,157,783,2197,2773,2147,626
7,2020-03-23,X20SDXSHA5011OW270,280,4,741,1646,2781,3636,2731,905
8,2020-03-30,X20SDXSHA5011OW270,90,0,649,1550,2967,3818,2917,901
9,2020-04-13,X20SDXSHA5011OW270,466,81,712,1641,3341,4140,3211,929


In [121]:
v2_stock_scsweek_pd.groupby(['weekkey']).sum().wms_stock_qty.mean()

7225.745762711865

In [49]:
v2_zero.groupby(['weekkey','scskey']).count().qty.reset_index()

,weekkey,scskey,qty
0,2019-11-18,X20SDXSHA5011OW230,3
1,2019-11-18,X20SDXSHA5011OW235,5
2,2019-11-18,X20SDXSHA5011OW240,2
3,2019-11-18,X20SDXSHA5011OW260,4
4,2019-11-18,X20SDXSHA5011OW265,4
5,2019-11-18,X20SDXSHA5011OW270,1
6,2019-11-25,X20SDXSHA5011OW230,1
7,2019-11-25,X20SDXSHA5011OW235,7
8,2019-11-25,X20SDXSHA5011OW240,3
9,2019-11-25,X20SDXSHA5011OW260,5


In [50]:
shop_weeks_buket_v2_df_dep.groupby(['weekkey','scskey']).sum().stock_qty.reset_index()

,weekkey,scskey,stock_qty
0,2019-11-18,X20SDXSHA5011OW230,108
1,2019-11-18,X20SDXSHA5011OW235,102
2,2019-11-18,X20SDXSHA5011OW240,101
3,2019-11-18,X20SDXSHA5011OW260,105
4,2019-11-18,X20SDXSHA5011OW265,108
5,2019-11-18,X20SDXSHA5011OW270,110
6,2019-11-25,X20SDXSHA5011OW230,114
7,2019-11-25,X20SDXSHA5011OW235,111
8,2019-11-25,X20SDXSHA5011OW240,123
9,2019-11-25,X20SDXSHA5011OW260,106


In [51]:
v2_stock_scsweek_pd.groupby(['weekkey','scskey']).sum().wms_stock_qty.reset_index()

,weekkey,scskey,wms_stock_qty
0,2019-11-18,X20SDXSHA5011OW230,404
1,2019-11-18,X20SDXSHA5011OW235,551
2,2019-11-18,X20SDXSHA5011OW240,911
3,2019-11-18,X20SDXSHA5011OW260,424
4,2019-11-18,X20SDXSHA5011OW265,423
5,2019-11-18,X20SDXSHA5011OW270,511
6,2019-11-25,X20SDXSHA5011OW230,394
7,2019-11-25,X20SDXSHA5011OW235,530
8,2019-11-25,X20SDXSHA5011OW240,885
9,2019-11-25,X20SDXSHA5011OW260,446


## 2019년 버킷디워커

In [52]:
shop_weeks_buket_df_dep = shop_weeks_buket_df[shop_weeks_buket_df['type_zone_nm']=='백화점']

In [53]:
shop_weeks_buket_df_dep

,weekkey,week_takeout_qty,takeout_qty,week_sale_qty,sale_qty,stock_qty,shopkey,ratio,scskey,shop_nm,shop_nm_sfdc,repr_shopcode,type_shop_nm,type_contract_nm,type_zone_nm,region_nm
0,2018-12-31,0,0,0,0,0,X10056,0.00000,X19SDXSH09911BE230,갤러리아진주,디스커버리 갤러리아 진주점,XH0247,일반매장,백화점특정,백화점,경남
1,2019-01-07,0,0,0,0,0,X10056,0.00000,X19SDXSH09911BE230,갤러리아진주,디스커버리 갤러리아 진주점,XH0247,일반매장,백화점특정,백화점,경남
2,2019-01-14,0,0,0,0,0,X10056,0.00000,X19SDXSH09911BE230,갤러리아진주,디스커버리 갤러리아 진주점,XH0247,일반매장,백화점특정,백화점,경남
3,2019-01-21,0,0,0,0,0,X10056,0.00000,X19SDXSH09911BE230,갤러리아진주,디스커버리 갤러리아 진주점,XH0247,일반매장,백화점특정,백화점,경남
4,2019-01-28,0,0,0,0,0,X10056,0.00000,X19SDXSH09911BE230,갤러리아진주,디스커버리 갤러리아 진주점,XH0247,일반매장,백화점특정,백화점,경남
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45363,2019-12-02,0,21,0,21,0,X10038,1.00000,X19SDXSH09911BE270,롯데창원,디스커버리 롯데 창원점,XH0237,일반매장,백화점특정,백화점,경남
45364,2019-12-09,0,21,0,21,0,X10038,1.00000,X19SDXSH09911BE270,롯데창원,디스커버리 롯데 창원점,XH0237,일반매장,백화점특정,백화점,경남
45365,2019-12-16,0,21,0,21,0,X10038,1.00000,X19SDXSH09911BE270,롯데창원,디스커버리 롯데 창원점,XH0237,일반매장,백화점특정,백화점,경남
45366,2019-12-23,0,21,0,21,0,X10038,1.00000,X19SDXSH09911BE270,롯데창원,디스커버리 롯데 창원점,XH0237,일반매장,백화점특정,백화점,경남


In [54]:
bk_shopkey_unique =  shop_weeks_buket_df_dep.shopkey.unique()
len(bk_shopkey_unique)

67

In [55]:
bk_scskey_unique =  shop_weeks_buket_df_dep.scskey.unique()
len(bk_scskey_unique)

6

In [56]:
shop_weeks_buket_df_dep = shop_weeks_buket_df_dep[shop_weeks_buket_df_dep['shopkey'] != 'X10047']

In [57]:
bk_shop_qty = pd.DataFrame()

for i in bk_shopkey_unique:
    uniqe_shop = shop_weeks_buket_df_dep[shop_weeks_buket_df_dep['shopkey']==i]
    for j in bk_scskey_unique:
        uniqe_shop_scskey = uniqe_shop[uniqe_shop['scskey']==j]
        ewm_value = make_ewm_arr(uniqe_shop_scskey, 2)
        uniqe_shop_scskey.insert(11, 'amount_2ewm', ewm_value)
        ewm_value = np.append([0],ewm_value,axis=0)
        uniqe_shop_scskey['amount_2ewm_week'] = ewm_value[:-1]
        for i in range(0,len(ewm_value)):
#             ewm_value[i] = round(ewm_value[i] , 0)+1
            ewm_value[i] = math.ceil(ewm_value[i])+1

        uniqe_shop_scskey['qty'] = ewm_value[:-1]
        bk_shop_qty = pd.concat([bk_shop_qty, uniqe_shop_scskey], axis=0)

<ipython-input-57-d32067fabf12>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uniqe_shop_scskey['amount_2ewm_week'] = ewm_value[:-1]
<ipython-input-57-d32067fabf12>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uniqe_shop_scskey['qty'] = ewm_value[:-1]


In [119]:
bk_shop_qty.groupby('weekkey').sum().stock_qty.mean()

1801.9622641509434

In [117]:
bk_shop_qty.groupby('weekkey').sum().stock_qty.to_csv('bk_shop_qty.csv')

In [58]:
bk_zero = bk_shop_qty[(bk_shop_qty['stock_qty']==0) & (bk_shop_qty['takeout_qty']>0)]

In [59]:
bk_zero

,weekkey,week_takeout_qty,takeout_qty,week_sale_qty,sale_qty,stock_qty,shopkey,ratio,scskey,shop_nm,shop_nm_sfdc,amount_2ewm,repr_shopcode,type_shop_nm,type_contract_nm,type_zone_nm,region_nm,amount_2ewm_week,qty
6,2019-02-11,1,1,0,1,0,X10056,0.08333,X19SDXSH09911BE230,갤러리아진주,디스커버리 갤러리아 진주점,0.22232,XH0247,일반매장,백화점특정,백화점,경남,0.66758,2.00000
15162,2019-01-28,-1,2,0,2,0,X10056,0.10526,X19SDXSH09911BE240,갤러리아진주,디스커버리 갤러리아 진주점,0.29752,XH0247,일반매장,백화점특정,백화점,경남,0.90000,2.00000
15163,2019-02-04,0,2,0,2,0,X10056,0.10526,X19SDXSH09911BE240,갤러리아진주,디스커버리 갤러리아 진주점,0.09890,XH0247,일반매장,백화점특정,백화점,경남,0.29752,2.00000
15164,2019-02-11,0,2,0,2,0,X10056,0.10526,X19SDXSH09911BE240,갤러리아진주,디스커버리 갤러리아 진주점,0.03294,XH0247,일반매장,백화점특정,백화점,경남,0.09890,2.00000
15166,2019-02-25,0,3,1,3,0,X10056,0.15789,X19SDXSH09911BE240,갤러리아진주,디스커버리 갤러리아 진주점,0.67036,XH0247,일반매장,백화점특정,백화점,경남,0.01098,2.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45363,2019-12-02,0,21,0,21,0,X10038,1.00000,X19SDXSH09911BE270,롯데창원,디스커버리 롯데 창원점,0.00007,XH0237,일반매장,백화점특정,백화점,경남,0.00020,2.00000
45364,2019-12-09,0,21,0,21,0,X10038,1.00000,X19SDXSH09911BE270,롯데창원,디스커버리 롯데 창원점,0.00002,XH0237,일반매장,백화점특정,백화점,경남,0.00007,2.00000
45365,2019-12-16,0,21,0,21,0,X10038,1.00000,X19SDXSH09911BE270,롯데창원,디스커버리 롯데 창원점,0.00001,XH0237,일반매장,백화점특정,백화점,경남,0.00002,2.00000
45366,2019-12-23,0,21,0,21,0,X10038,1.00000,X19SDXSH09911BE270,롯데창원,디스커버리 롯데 창원점,0.00000,XH0237,일반매장,백화점특정,백화점,경남,0.00001,2.00000


In [60]:
len(bk_zero)

2225

In [61]:
# 재고에서 0 이 지워진 부분 
len(bk_zero[bk_zero.qty - bk_zero.week_sale_qty>0])

1863

In [62]:
# 아직 재고가 0 이하인 부분
len(bk_zero[bk_zero.qty - bk_zero.week_sale_qty<=0])

362

In [63]:
# zero 인 지점
bk_zero.groupby(['weekkey','scskey']).count().qty.reset_index()

,weekkey,scskey,qty
0,2018-12-31,X19SDXSH09911BE235,1
1,2018-12-31,X19SDXSH09911BE240,1
2,2018-12-31,X19SDXSH09911BE260,2
3,2018-12-31,X19SDXSH09911BE270,1
4,2019-01-07,X19SDXSH09911BE230,1
5,2019-01-07,X19SDXSH09911BE235,3
6,2019-01-07,X19SDXSH09911BE240,2
7,2019-01-07,X19SDXSH09911BE260,6
8,2019-01-07,X19SDXSH09911BE265,2
9,2019-01-07,X19SDXSH09911BE270,4


In [64]:
# stock_qty 재고 개수
shop_weeks_buket_df_dep.groupby(['weekkey','scskey']).sum().stock_qty.reset_index()

,weekkey,scskey,stock_qty
0,2018-12-31,X19SDXSH09911BE230,15
1,2018-12-31,X19SDXSH09911BE235,63
2,2018-12-31,X19SDXSH09911BE240,79
3,2018-12-31,X19SDXSH09911BE260,62
4,2018-12-31,X19SDXSH09911BE265,17
5,2018-12-31,X19SDXSH09911BE270,63
6,2019-01-07,X19SDXSH09911BE230,12
7,2019-01-07,X19SDXSH09911BE235,59
8,2019-01-07,X19SDXSH09911BE240,74
9,2019-01-07,X19SDXSH09911BE260,58


In [65]:
bk_stock_scsweek_pd.groupby(['weekkey','scskey']).sum().wms_stock_qty.reset_index()

,weekkey,scskey,wms_stock_qty
0,2018-12-31,X19SDXSH09911BE230,0
1,2018-12-31,X19SDXSH09911BE235,1
2,2018-12-31,X19SDXSH09911BE240,1
3,2018-12-31,X19SDXSH09911BE260,1
4,2018-12-31,X19SDXSH09911BE265,0
5,2018-12-31,X19SDXSH09911BE270,1
6,2019-01-07,X19SDXSH09911BE230,0
7,2019-01-07,X19SDXSH09911BE235,1
8,2019-01-07,X19SDXSH09911BE240,1
9,2019-01-07,X19SDXSH09911BE260,1


In [66]:
bk_zero_modi = bk_zero[bk_zero.qty - bk_zero.week_sale_qty<=0].groupby(['weekkey']).count().qty.reset_index()

In [67]:
bk_zero_ori = bk_zero.groupby(['weekkey']).count().qty.reset_index()

In [68]:
bk_zero_modi_frame = shop_weeks_buket_df_dep[:53][['weekkey','stock_qty']]
bk_zero_modi_frame.stock_qty = 0

for i in range(0,len(bk_zero_modi_frame)):
    for j in range(0,len(bk_zero_modi)):
        if bk_zero_modi_frame.iloc[i].weekkey == bk_zero_modi.iloc[j].weekkey:
            bk_zero_modi_frame.iloc[i] = [bk_zero_modi.iloc[j].weekkey , bk_zero_modi.iloc[j].qty]
bk_zero_modi_frame.to_csv('bk_zero_modi_frame.csv')

In [69]:
bk_zero_ori_frame = shop_weeks_buket_df_dep[:53][['weekkey','stock_qty']]
bk_zero_ori_frame.stock_qty = 0

for i in range(0,len(bk_zero_ori_frame)):
    for j in range(0,len(bk_zero_ori)):
        if bk_zero_ori_frame.iloc[i].weekkey == bk_zero_ori.iloc[j].weekkey:
            bk_zero_ori_frame.iloc[i] = [bk_zero_ori.iloc[j].weekkey , bk_zero_ori.iloc[j].qty]
bk_zero_ori_frame.to_csv('bk_zero_ori_frame.csv')

In [70]:
bk_merge_outer = pd.merge(bk_zero_ori_frame,bk_zero_modi_frame, how='outer',on='weekkey')
bk_merge_outer.to_csv('bk_merge_outer.csv')

In [71]:
bk_merge_outer

,weekkey,stock_qty_x,stock_qty_y
0,2018-12-31,5,5
1,2019-01-07,18,16
2,2019-01-14,76,74
3,2019-01-21,109,32
4,2019-01-28,103,16
5,2019-02-04,98,26
6,2019-02-11,124,7
7,2019-02-18,103,20
8,2019-02-25,150,42
9,2019-03-04,64,18


In [72]:
v2_shop_qty[v2_shop_qty.shop_nm == '현대판교']

,weekkey,week_takeout_qty,takeout_qty,week_sale_qty,sale_qty,stock_qty,shopkey,ratio,scskey,shop_nm,shop_nm_sfdc,amount_2ewm,repr_shopcode,type_shop_nm,type_contract_nm,type_zone_nm,region_nm,amount_2ewm_week,qty
1711,2019-11-18,1,1,0,0,1,X10068,0.00000,X20SDXSHA5011OW230,현대판교,디스커버리 현대 판교점,0.00000,XH0257,일반매장,백화점특정,백화점,경기,0.00000,1.00000
1712,2019-11-25,0,1,0,0,1,X10068,0.00000,X20SDXSHA5011OW230,현대판교,디스커버리 현대 판교점,0.00000,XH0257,일반매장,백화점특정,백화점,경기,0.00000,1.00000
1713,2019-12-02,0,1,0,0,1,X10068,0.00000,X20SDXSHA5011OW230,현대판교,디스커버리 현대 판교점,0.00000,XH0257,일반매장,백화점특정,백화점,경기,0.00000,1.00000
1714,2019-12-09,0,1,0,0,1,X10068,0.00000,X20SDXSHA5011OW230,현대판교,디스커버리 현대 판교점,0.00000,XH0257,일반매장,백화점특정,백화점,경기,0.00000,1.00000
1715,2019-12-16,15,16,15,15,1,X10068,0.57692,X20SDXSHA5011OW230,현대판교,디스커버리 현대 판교점,10.04132,XH0257,일반매장,백화점특정,백화점,경기,0.00000,1.00000
1716,2019-12-23,0,16,1,16,0,X10068,0.61538,X20SDXSHA5011OW230,현대판교,디스커버리 현대 판교점,4.00549,XH0257,일반매장,백화점특정,백화점,경기,10.04132,12.00000
1717,2019-12-30,4,20,0,16,4,X10068,0.61538,X20SDXSHA5011OW230,현대판교,디스커버리 현대 판교점,1.33394,XH0257,일반매장,백화점특정,백화점,경기,4.00549,6.00000
1718,2020-01-06,0,20,0,16,4,X10068,0.61538,X20SDXSHA5011OW230,현대판교,디스커버리 현대 판교점,0.44451,XH0257,일반매장,백화점특정,백화점,경기,1.33394,3.00000
1719,2020-01-13,0,20,1,17,3,X10068,0.65385,X20SDXSHA5011OW230,현대판교,디스커버리 현대 판교점,0.81486,XH0257,일반매장,백화점특정,백화점,경기,0.44451,2.00000
1720,2020-01-20,2,22,0,17,5,X10068,0.65385,X20SDXSHA5011OW230,현대판교,디스커버리 현대 판교점,0.27161,XH0257,일반매장,백화점특정,백화점,경기,0.81486,2.00000


In [131]:
sum(v2_shop_qty[v2_shop_qty['shop_nm']=='갤러리아진주'].stock_qty - v2_shop_qty[v2_shop_qty['shop_nm']=='갤러리아진주'].qty)

257.0

In [135]:
test1 = v2_shop_qty.copy()

In [144]:
test1['stock_qty'] = test1.stock_qty - test1.qty

In [145]:
test1.groupby('weekkey').sum().stock_qty

weekkey
2019-11-18    238.00000
2019-11-25    198.00000
2019-12-02    146.00000
2019-12-09     65.00000
2019-12-16     80.00000
2019-12-23    -60.00000
2019-12-30     -4.00000
2020-01-06     20.00000
2020-01-13    216.00000
2020-01-20    475.00000
2020-01-27    548.00000
2020-02-03    515.00000
2020-02-10    393.00000
2020-02-17    420.00000
2020-02-24    531.00000
2020-03-02    565.00000
2020-03-09    490.00000
2020-03-16    495.00000
2020-03-23    603.00000
2020-03-30    684.00000
2020-04-06    713.00000
2020-04-13    830.00000
2020-04-20    886.00000
2020-04-27    859.00000
2020-05-04   1031.00000
2020-05-11   1088.00000
2020-05-18   1134.00000
2020-05-25   1069.00000
2020-06-01   1198.00000
2020-06-08   1228.00000
2020-06-15   1260.00000
2020-06-22   1280.00000
2020-06-29   1295.00000
2020-07-06   1307.00000
2020-07-13   1313.00000
2020-07-20   1316.00000
2020-07-27   1338.00000
2020-08-03   1352.00000
2020-08-10   1337.00000
2020-08-17   1363.00000
2020-08-24   1381.00000
2020-08-

In [148]:
mae(test1.week_takeout_qty,test1.qty)

1.6996233521657251

In [149]:
mae(test1.week_takeout_qty,test1.qty)

1.6996233521657251

In [159]:
mae(test1.week_takeout_qty[1:],test1.stock_qty[:-1])

2.498908530582545

In [165]:
shopkey_unique = test1.shopkey.unique()
scskey_unique = test1.scskey.unique()
mae_list=[]
for i in shopkey_unique:
    for j in scskey_unique:
#         print(test1[(test1['shopkey']==i) & (test1['scskey']==j)].week_takeout_qty[1:])
        mae_list.append(mae( test1[(test1['shopkey']==i) & (test1['scskey']==j)].week_takeout_qty[1:] ,test1[(test1['shopkey']==i) & (test1['scskey']==j)].stock_qty[:-1] ))
#         break
#     break
mae_list
        
    

1     0
2     2
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    3
11    0
12    0
13    0
14    0
15    0
16    0
17    1
18    0
19    0
20    0
21    0
22    0
23    0
24    0
25    0
26    0
27    1
28    0
29    0
30    1
31    0
32    0
33    0
34    0
35    0
36    0
37    0
38    0
39    0
40    0
41    0
42    0
43    0
44   -2
45    0
46    0
47    0
48    0
49    0
50    0
51    1
52    2
53    0
54    0
55    0
56    0
57    0
58    2
Name: week_takeout_qty, dtype: int64
8438    0
8439    1
8440    0
8441    0
8442    1
8443    1
8444    0
8445    0
8446    0
8447    2
8448    0
8449    0
8450    0
8451    0
8452    0
8453    0
8454    0
8455    0
8456    0
8457    0
8458    1
8459    0
8460    0
8461    0
8462    0
8463    1
8464    0
8465    0
8466    0
8467    0
8468    0
8469    0
8470   -1
8471    0
8472    0
8473    0
8474    0
8475    0
8476    0
8477    0
8478    0
8479    0
8480    0
8481    1
8482    0
8483    0
8484    4
8485   -1
8486    0
8487    0

17701    3
17702    5
17703    2
17704    3
17705    6
17706    3
17707    4
17708    4
17709    3
17710    2
17711    8
17712    5
17713    8
17714    0
17715    2
17716    3
17717    4
17718    5
17719    5
17720    4
17721    5
17722    4
17723    1
17724    2
17725    2
17726    1
17727    1
17728    2
17729    2
17730    0
17731    1
17732    1
17733    0
17734    0
17735    1
17736    1
17737    3
17738    3
17739    1
17740    1
17741    0
17742    3
17743    1
17744    1
17745    2
17746    4
17747    2
17748    3
17749    0
17750    4
17751    2
17752    3
17753    2
17754    4
17755    1
17756    0
17757    0
17758    0
Name: week_takeout_qty, dtype: int64
26138    1
26139    3
26140    0
26141    2
26142    1
26143    0
26144    1
26145    2
26146    2
26147    1
26148    3
26149    4
26150    5
26151    1
26152    2
26153    3
26154    2
26155    1
26156    0
26157    0
26158    2
26159    0
26160    0
26161    2
26162    1
26163    0
26164    2
26165    1
26166    0
26167 

9913    0
9914    0
9915    0
9916    1
9917    0
9918    0
9919    0
9920    0
9921    0
9922    0
9923    0
9924    0
9925    1
9926    0
9927    1
9928    3
9929    1
9930    0
9931    0
9932    0
9933    1
9934    1
9935    0
9936    0
9937    0
9938    1
9939    1
9940    0
9941    0
9942    0
9943    0
9944    0
9945    0
9946    0
9947    0
9948    0
9949    0
9950    0
9951    0
9952    0
9953    0
9954    0
9955    0
9956    1
9957    0
9958    0
9959    0
9960    0
9961    0
9962    1
9963    0
9964    1
9965    1
9966    1
9967    0
9968    0
9969    0
9970    0
Name: week_takeout_qty, dtype: int64
18350    2
18351    0
18352    0
18353    0
18354    1
18355    0
18356    0
18357    1
18358    0
18359    0
18360    0
18361    0
18362    1
18363    0
18364    0
18365    0
18366    3
18367    2
18368    1
18369    2
18370    1
18371    0
18372    0
18373    3
18374    0
18375    0
18376    0
18377    0
18378    0
18379    0
18380    0
18381    0
18382    0
18383    0
18384    

18527    0
18528    0
18529    0
18530    2
18531    0
18532    1
18533    2
18534    1
18535    6
18536    1
18537    1
18538    3
18539   -1
18540    0
18541    0
18542    1
18543    1
18544    0
18545    1
18546    1
18547    1
18548    0
18549    0
18550    0
18551    0
18552    0
18553    0
18554    1
18555    0
18556    0
18557    0
18558    0
18559    1
18560    0
18561    0
18562    0
18563    0
18564    0
18565    1
18566    1
18567    0
18568    0
18569    2
18570    0
18571    0
18572    1
18573    1
18574    0
18575    2
18576    0
18577    0
18578    0
18579    0
18580    0
18581    0
18582    0
18583    0
18584    0
Name: week_takeout_qty, dtype: int64
26964    0
26965    0
26966    1
26967    0
26968    0
26969    0
26970    0
26971    0
26972    0
26973    0
26974    0
26975    0
26976    0
26977    0
26978    0
26979    0
26980    0
26981    1
26982    0
26983    0
26984    0
26985    0
26986    3
26987    2
26988    2
26989    0
26990    0
26991    0
26992    0
26993 

44074    0
44075    2
44076    0
44077    0
44078    0
44079    0
44080    1
44081    2
44082    0
44083    1
44084    0
44085    0
44086   -1
44087    1
44088    0
44089    4
44090    0
44091    0
44092    0
44093    0
44094    0
44095    0
44096    0
44097    1
44098    0
44099    0
44100    0
44101    1
44102    1
44103    1
44104    0
44105    0
44106    0
44107    0
44108    0
44109    1
44110    0
44111    0
44112    0
44113    0
44114    0
44115    1
44116    0
44117    0
44118    0
44119    1
44120    0
44121    0
44122    0
44123    1
44124    1
44125    1
44126    1
44127    0
44128    0
44129    0
44130    0
44131    0
Name: week_takeout_qty, dtype: int64
2184    0
2185    0
2186    0
2187    0
2188    0
2189    0
2190    0
2191    0
2192    0
2193    0
2194    3
2195    0
2196    0
2197    0
2198    2
2199    0
2200    0
2201    0
2202    0
2203    0
2204    0
2205    0
2206    0
2207    0
2208    0
2209    0
2210   -3
2211    3
2212    0
2213    0
2214    0
2215    0
2216 

...

14633    2
14634    0
14635    3
14636    0
14637    1
14638    1
14639    2
14640    0
14641    2
14642    3
14643    0
14644    2
14645    1
14646    1
14647    1
14648    1
14649    0
14650    0
14651    1
14652    0
14653    0
14654    0
14655    1
14656    0
14657    0
14658    0
14659    0
14660    0
14661    0
14662    0
14663    0
14664    1
14665    0
14666    0
14667    0
14668    0
14669    0
14670    0
14671    0
14672    0
14673    0
14674    0
14675    0
14676    0
14677    0
14678    0
14679    0
14680    0
14681    0
14682    0
14683    0
14684    0
14685    0
14686    0
14687    0
14688    0
14689    0
14690    0
Name: week_takeout_qty, dtype: int64
23070    3
23071    0
23072    0
23073    0
23074    2
23075    0
23076    0
23077    1
23078    3
23079    0
23080    1
23081    2
23082    0
23083    1
23084    0
23085    1
23086    0
23087    1
23088    0
23089    0
23090    1
23091    0
23092    0
23093    0
23094    0
23095    0
23096    0
23097    1
23098    1
23099 

32392    0
32393    0
32394    3
32395    0
32396    1
32397    1
32398    1
32399    3
32400    3
32401    1
32402    1
32403    0
32404    1
32405    0
32406    1
32407    1
32408   -1
32409    0
32410    1
32411    0
32412    0
32413    1
32414    0
32415    1
32416    0
32417    0
32418   -1
32419    0
32420    0
32421    0
32422    0
32423    0
32424    0
32425    0
32426    0
32427    1
32428    0
32429    0
32430    0
32431    0
32432    0
32433    0
32434    0
32435    0
32436    0
32437    1
32438    1
32439    0
32440    0
32441    0
32442    1
32443    0
32444    0
32445    0
32446    0
32447   -2
32448   -1
32449   -2
Name: week_takeout_qty, dtype: int64
40829    0
40830    0
40831    0
40832    2
40833    0
40834    0
40835    1
40836    1
40837    3
40838    0
40839    0
40840    0
40841    0
40842    3
40843    2
40844    0
40845    0
40846    2
40847    0
40848    0
40849    0
40850    0
40851    1
40852    2
40853    0
40854    0
40855    1
40856    0
40857    0
40858 

Name: week_takeout_qty, dtype: int64
41537    0
41538    0
41539    3
41540    2
41541    2
41542    0
41543    0
41544    0
41545    0
41546    0
41547    2
41548    1
41549    3
41550    0
41551    1
41552    0
41553   -1
41554    0
41555    1
41556    0
41557    0
41558    1
41559    1
41560    0
41561    0
41562    0
41563    1
41564    0
41565    0
41566    0
41567    1
41568    0
41569    0
41570    1
41571    0
41572    0
41573    0
41574    0
41575    0
41576    0
41577    0
41578    1
41579    0
41580    0
41581    0
41582    0
41583    1
41584    0
41585    1
41586    0
41587    0
41588    0
41589    0
41590    0
41591    0
41592    0
41593    0
41594    0
Name: week_takeout_qty, dtype: int64
49974    0
49975    0
49976    0
49977    0
49978    1
49979    0
49980    1
49981    2
49982    3
49983    0
49984    3
49985    2
49986    0
49987    1
49988   -1
49989    0
49990    0
49991    1
49992    0
49993    0
49994    1
49995    1
49996    1
49997    0
49998    0
49999    0
50

Name: week_takeout_qty, dtype: int64
25076    0
25077    2
25078    2
25079    1
25080    1
25081    0
25082    1
25083    1
25084    0
25085    1
25086    2
25087    0
25088    2
25089    1
25090    0
25091    0
25092    0
25093    0
25094    0
25095    2
25096    0
25097    1
25098    1
25099    0
25100    0
25101    0
25102    0
25103    0
25104    0
25105    0
25106    0
25107    0
25108    2
25109    0
25110    0
25111    0
25112    0
25113    0
25114    0
25115    0
25116    0
25117    0
25118    0
25119    0
25120    1
25121    1
25122    1
25123    1
25124    0
25125    0
25126    1
25127    0
25128    0
25129    0
25130    0
25131    0
25132    0
25133    0
Name: week_takeout_qty, dtype: int64
33513    2
33514    0
33515    0
33516    0
33517    3
33518    0
33519    0
33520    2
33521    0
33522    1
33523    0
33524    0
33525    0
33526    1
33527    0
33528    0
33529    0
33530    0
33531    0
33532    0
33533    1
33534    0
33535    0
33536    0
33537    0
33538    0
33

[1.0689655172413792,
 1.1206896551724137,
 1.293103448275862,
 1.1896551724137931,
 1.6724137931034482,
 1.0,
 2.2586206896551726,
 3.5172413793103448,
 3.1724137931034484,
 0.9137931034482759,
 2.5344827586206895,
 1.8620689655172413,
 4.379310344827586,
 3.4482758620689653,
 2.603448275862069,
 3.206896551724138,
 3.1206896551724137,
 4.086206896551724,
 1.4482758620689655,
 2.4827586206896552,
 2.5517241379310347,
 1.0517241379310345,
 2.7413793103448274,
 1.7241379310344827,
 2.396551724137931,
 3.810344827586207,
 3.810344827586207,
 2.1379310344827585,
 2.1206896551724137,
 2.3620689655172415,
 1.7413793103448276,
 1.603448275862069,
 1.2241379310344827,
 0.896551724137931,
 1.3275862068965518,
 1.1551724137931034,
 1.5172413793103448,
 3.396551724137931,
 1.9482758620689655,
 1.103448275862069,
 2.086206896551724,
 1.5517241379310345,
 1.9655172413793103,
 2.0344827586206895,
 2.793103448275862,
 2.6551724137931036,
 2.5517241379310347,
 1.6896551724137931,
 1.6206896551724137,


In [167]:
np.mean(mae_list)

2.5141501219087425